# 7.3 RESTful API设计与实现

## 📚 用途说明
本笔记本深入讲解RESTful API设计原则和最佳实践，重点掌握生产级API的设计模式。包括分页、过滤、版本控制、速率限制、缓存等核心功能，为构建可扩展的LangChain Web应用提供完整解决方案。

### 🎯 学习目标
- 掌握RESTful API设计原则和约束
- 实现分页、排序、过滤等数据查询功能
- 设计API版本控制策略
- 构建速率限制和缓存机制
- 实现HATEOAS和超媒体导航
- 为LangChain应用设计生产级API

### ⚡ 前置要求
- HTTP协议基础和状态码理解
- FastAPI框架基础使用
- 异步编程和数据库操作
- 数据验证和错误处理

### 🔄 LangChain关联
RESTful API是LangChain生产应用的核心：
- 聊天历史分页：管理大量对话记录
- 文档检索过滤：精确的RAG查询控制
- 模型版本管理：支持多版本AI模型
- 缓存优化：减少AI API调用成本

### 📦 依赖安装
```bash
pip install fastapi uvicorn pydantic slowapi redis
pip install aiofiles python-multipart
```

In [ ]:
# 导入必要依赖
import asyncio
import json
import time
import uuid
import hashlib
from typing import Dict, List, Optional, Any, Union, Tuple
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from urllib.parse import urlparse, parse_qs
import math

# FastAPI相关导入
try:
    from fastapi import FastAPI, HTTPException, Depends, Query, Header, Request, Response
    from fastapi.responses import JSONResponse
    from fastapi.middleware.cors import CORSMiddleware
    from pydantic import BaseModel, Field, validator
    from slowapi import Limiter, _rate_limit_exceeded_handler
    from slowapi.util import get_remote_address
    from slowapi.errors import RateLimitExceeded
    
    FASTAPI_AVAILABLE = True
    print("✅ RESTful API依赖导入成功")
except ImportError as e:
    FASTAPI_AVAILABLE = False
    print(f"❌ 依赖缺失: {e}")
    print("请运行: pip install fastapi uvicorn pydantic slowapi")

print("\n🌐 RESTful API设计与实现")
print("=" * 50)
print(f"📝 学习目标: 掌握生产级RESTful API设计")
print(f"⚡ 核心特性: 分页、过滤、版本控制、缓存、限流")

## 1. RESTful API设计原则

### 1.1 REST架构约束

In [ ]:
print(f"\n🏗️ 1. RESTful API设计原则:")

class RESTPrinciples:
    """REST架构原则说明"""
    
    principles = {
        "客户端-服务器分离": {
            "description": "客户端和服务器职责分离，提高可移植性和可扩展性",
            "example": "前端应用与后端API独立部署"
        },
        "无状态": {
            "description": "每个请求包含所有必要信息，服务器不保存客户端状态",
            "example": "使用JWT token进行身份验证"
        },
        "可缓存": {
            "description": "响应数据应标记为可缓存或不可缓存",
            "example": "使用Cache-Control头部控制缓存"
        },
        "统一接口": {
            "description": "使用标准HTTP方法和状态码，统一的资源命名",
            "example": "GET /users, POST /users, PUT /users/{id}"
        },
        "分层系统": {
            "description": "系统由多层组成，每层只了解相邻层",
            "example": "负载均衡器 -> API网关 -> 应用服务 -> 数据库"
        },
        "按需代码": {
            "description": "服务器可以临时扩展客户端功能",
            "example": "返回JavaScript代码或小程序"
        }
    }
    
    @classmethod
    def print_principles(cls):
        """打印REST原则"""
        print(f"   📋 REST架构约束:")
        for principle, info in cls.principles.items():
            print(f"\n      🔹 {principle}:")
            print(f"         描述: {info['description']}")
            print(f"         示例: {info['example']}")

# RESTful API设计最佳实践
class RESTfulBestPractices:
    """RESTful API最佳实践"""
    
    url_design = {
        "资源命名": "使用名词而非动词，复数形式表示集合",
        "层级关系": "使用URL路径表示资源层级关系",
        "查询参数": "使用查询参数进行过滤、排序、分页",
        "版本控制": "在URL中包含版本号或使用头部"
    }
    
    http_methods = {
        "GET": "获取资源，安全且幂等",
        "POST": "创建资源，非安全且非幂等",
        "PUT": "完整更新资源，非安全但幂等",
        "PATCH": "部分更新资源，非安全且非幂等",
        "DELETE": "删除资源，非安全但幂等"
    }
    
    status_codes = {
        "2xx": "成功响应",
        "3xx": "重定向",
        "4xx": "客户端错误",
        "5xx": "服务器错误"
    }
    
    @classmethod
    def print_best_practices(cls):
        """打印最佳实践"""
        print(f"\n   🎯 API设计最佳实践:")
        
        print(f"\n      📍 URL设计:")
        for rule, desc in cls.url_design.items():
            print(f"         - {rule}: {desc}")
        
        print(f"\n      🔧 HTTP方法:")
        for method, desc in cls.http_methods.items():
            print(f"         - {method}: {desc}")
        
        print(f"\n      📊 状态码分类:")
        for code_range, desc in cls.status_codes.items():
            print(f"         - {code_range}: {desc}")

# 打印设计原则
RESTPrinciples.print_principles()
RESTfulBestPractices.print_best_practices()

### 1.2 资源建模和URL设计

In [ ]:
print(f"\n🎨 2. 资源建模和URL设计:")

# LangChain资源模型设计
class LangChainResourceModel:
    """LangChain应用资源模型"""
    
    resources = {
        "conversations": {
            "description": "对话会话资源",
            "endpoints": {
                "GET /api/v1/conversations": "获取对话列表",
                "POST /api/v1/conversations": "创建新对话",
                "GET /api/v1/conversations/{id}": "获取指定对话",
                "PUT /api/v1/conversations/{id}": "更新对话信息",
                "DELETE /api/v1/conversations/{id}": "删除对话",
                "GET /api/v1/conversations/{id}/messages": "获取对话消息"
            }
        },
        "documents": {
            "description": "文档资源（用于RAG）",
            "endpoints": {
                "GET /api/v1/documents": "获取文档列表",
                "POST /api/v1/documents": "上传文档",
                "GET /api/v1/documents/{id}": "获取文档内容",
                "PUT /api/v1/documents/{id}": "更新文档",
                "DELETE /api/v1/documents/{id}": "删除文档",
                "POST /api/v1/documents/search": "文档搜索"
            }
        },
        "models": {
            "description": "AI模型资源",
            "endpoints": {
                "GET /api/v1/models": "获取可用模型列表",
                "GET /api/v1/models/{id}": "获取模型详细信息",
                "POST /api/v1/models/{id}/chat": "模型聊天",
                "POST /api/v1/models/{id}/embeddings": "生成嵌入向量"
            }
        },
        "users": {
            "description": "用户资源",
            "endpoints": {
                "GET /api/v1/users": "获取用户列表",
                "POST /api/v1/users": "创建用户",
                "GET /api/v1/users/{id}": "获取用户信息",
                "PUT /api/v1/users/{id}": "更新用户信息",
                "DELETE /api/v1/users/{id}": "删除用户",
                "GET /api/v1/users/{id}/conversations": "获取用户对话"
            }
        }
    }
    
    @classmethod
    def print_resource_model(cls):
        """打印资源模型"""
        print(f"   📊 LangChain资源模型设计:")
        for resource_name, resource_info in cls.resources.items():
            print(f"\n      🗂️ {resource_name.upper()} - {resource_info['description']}:")
            for endpoint, description in resource_info['endpoints'].items():
                print(f"         {endpoint:<35} : {description}")

# URL设计验证器
class URLDesignValidator:
    """URL设计验证器"""
    
    @staticmethod
    def validate_resource_name(name: str) -> bool:
        """验证资源名称"""
        # 应该使用小写名词，复数形式
        return name.islower() and name.isalpha() and not name.endswith('s') or name.endswith('s')
    
    @staticmethod
    def validate_http_method(url: str, method: str) -> bool:
        """验证HTTP方法使用"""
        valid_methods = ['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
        return method.upper() in valid_methods
    
    @staticmethod
    def suggest_url_structure(resource: str, action: str = None) -> str:
        """建议URL结构"""
        base_url = f"/api/v1/{resource}s"  # 复数形式
        
        if action == "create":
            return f"POST {base_url}"
        elif action == "list":
            return f"GET {base_url}"
        elif action == "detail":
            return f"GET {base_url}/{{id}}"
        elif action == "update":
            return f"PUT {base_url}/{{id}}"
        elif action == "delete":
            return f"DELETE {base_url}/{{id}}"
        else:
            return base_url

# 打印资源模型
LangChainResourceModel.print_resource_model()

print(f"\n   🔍 URL设计验证:")
validator = URLDesignValidator()

# 测试URL设计
test_resources = ["conversation", "document", "model", "user"]
for resource in test_resources:
    is_valid = validator.validate_resource_name(resource)
    detail_url = validator.suggest_url_structure(resource, "detail")
    list_url = validator.suggest_url_structure(resource, "list")
    print(f"      {resource:<12} : {'✅' if is_valid else '❌'} {list_url}, {detail_url}")

## 2. 分页和过滤实现

### 2.1 分页策略和实现

In [ ]:
print(f"\n📄 3. 分页和过滤实现:")

# 分页参数模型
class PaginationParams(BaseModel):
    """分页参数模型"""
    page: int = Field(1, ge=1, description="页码，从1开始")
    size: int = Field(20, ge=1, le=100, description="每页大小，最大100")
    
    @property
    def offset(self) -> int:
        """计算偏移量"""
        return (self.page - 1) * self.size
    
    @property
    def limit(self) -> int:
        """获取限制数量"""
        return self.size

# 分页响应模型
class PaginationMeta(BaseModel):
    """分页元数据"""
    page: int = Field(..., description="当前页码")
    size: int = Field(..., description="每页大小")
    total: int = Field(..., description="总记录数")
    pages: int = Field(..., description="总页数")
    has_next: bool = Field(..., description="是否有下一页")
    has_prev: bool = Field(..., description="是否有上一页")

class PaginatedResponse(BaseModel, Generic[TypeVar('T')]):
    """分页响应模型"""
    data: List[Any] = Field(..., description="数据列表")
    meta: PaginationMeta = Field(..., description="分页元数据")
    links: Dict[str, str] = Field(..., description="导航链接")

# 分页工具类
class PaginationHelper:
    """分页工具类"""
    
    @staticmethod
    def create_pagination_meta(page: int, size: int, total: int) -> PaginationMeta:
        """创建分页元数据"""
        pages = math.ceil(total / size) if size > 0 else 0
        
        return PaginationMeta(
            page=page,
            size=size,
            total=total,
            pages=pages,
            has_next=page < pages,
            has_prev=page > 1
        )
    
    @staticmethod
    def create_navigation_links(base_url: str, page: int, size: int, total: int) -> Dict[str, str]:
        """创建导航链接（HATEOAS）"""
        pages = math.ceil(total / size) if size > 0 else 0
        links = {}
        
        # 当前页链接
        links["self"] = f"{base_url}?page={page}&size={size}"
        
        # 第一页链接
        if page > 1:
            links["first"] = f"{base_url}?page=1&size={size}"
        
        # 上一页链接
        if page > 1:
            links["prev"] = f"{base_url}?page={page-1}&size={size}"
        
        # 下一页链接
        if page < pages:
            links["next"] = f"{base_url}?page={page+1}&size={size}"
        
        # 最后一页链接
        if page < pages:
            links["last"] = f"{base_url}?page={pages}&size={size}"
        
        return links
    
    @staticmethod
    def paginate_data(data: List[Any], page: int, size: int) -> List[Any]:
        """对数据进行分页"""
        offset = (page - 1) * size
        return data[offset:offset + size]

print(f"   ✅ 分页模型和工具类创建完成")
print(f"      - PaginationParams: 分页参数验证")
print(f"      - PaginationMeta: 分页元数据")
print(f"      - PaginatedResponse: 统一分页响应格式")
print(f"      - PaginationHelper: 分页计算工具")

### 2.2 过滤和排序实现

In [ ]:
print(f"\n🔍 4. 过滤和排序实现:")

# 过滤参数模型
class FilterParams(BaseModel):
    """过滤参数模型"""
    search: Optional[str] = Field(None, description="搜索关键词")
    status: Optional[str] = Field(None, description="状态过滤")
    category: Optional[str] = Field(None, description="分类过滤")
    date_from: Optional[datetime] = Field(None, description="开始日期")
    date_to: Optional[datetime] = Field(None, description="结束日期")
    tags: Optional[List[str]] = Field(None, description="标签过滤")

# 排序参数模型
class SortParams(BaseModel):
    """排序参数模型"""
    sort_by: str = Field("created_at", description="排序字段")
    sort_order: str = Field("desc", regex="^(asc|desc)$", description="排序方向")
    
    @validator('sort_order')
    def validate_sort_order(cls, v):
        if v not in ['asc', 'desc']:
            raise ValueError('排序方向必须是 asc 或 desc')
        return v

# 查询参数组合
class QueryParams(BaseModel):
    """综合查询参数"""
    pagination: PaginationParams = Field(default_factory=PaginationParams)
    filter: FilterParams = Field(default_factory=FilterParams)
    sort: SortParams = Field(default_factory=SortParams)

# 数据过滤工具类
class DataFilter:
    """数据过滤工具类"""
    
    @staticmethod
    def filter_conversations(data: List[Dict], filter_params: FilterParams) -> List[Dict]:
        """过滤对话数据"""
        filtered_data = data.copy()
        
        # 搜索过滤
        if filter_params.search:
            search_term = filter_params.search.lower()
            filtered_data = [
                item for item in filtered_data
                if search_term in item.get('title', '').lower() or 
                   search_term in item.get('description', '').lower()
            ]
        
        # 状态过滤
        if filter_params.status:
            filtered_data = [
                item for item in filtered_data
                if item.get('status') == filter_params.status
            ]
        
        # 分类过滤
        if filter_params.category:
            filtered_data = [
                item for item in filtered_data
                if item.get('category') == filter_params.category
            ]
        
        # 日期范围过滤
        if filter_params.date_from:
            filtered_data = [
                item for item in filtered_data
                if datetime.fromisoformat(item.get('created_at')) >= filter_params.date_from
            ]
        
        if filter_params.date_to:
            filtered_data = [
                item for item in filtered_data
                if datetime.fromisoformat(item.get('created_at')) <= filter_params.date_to
            ]
        
        # 标签过滤
        if filter_params.tags:
            filtered_data = [
                item for item in filtered_data
                if any(tag in item.get('tags', []) for tag in filter_params.tags)
            ]
        
        return filtered_data
    
    @staticmethod
    def sort_data(data: List[Dict], sort_params: SortParams) -> List[Dict]:
        """排序数据"""
        reverse = sort_params.sort_order == 'desc'
        
        try:
            return sorted(
                data,
                key=lambda x: x.get(sort_params.sort_by, ''),
                reverse=reverse
            )
        except (TypeError, KeyError):
            # 如果排序失败，返回原始数据
            return data

# 创建示例数据
def create_sample_conversations(count: int = 50) -> List[Dict]:
    """创建示例对话数据"""
    conversations = []
    statuses = ['active', 'archived', 'pending']
    categories = ['general', 'technical', 'support', 'sales']
    
    for i in range(count):
        created_at = datetime.now() - timedelta(days=i % 30)
        conversation = {
            'id': i + 1,
            'title': f'对话主题 {i + 1}',
            'description': f'这是关于{i + 1}号对话的详细描述',
            'status': statuses[i % len(statuses)],
            'category': categories[i % len(categories)],
            'created_at': created_at.isoformat(),
            'message_count': (i % 20) + 1,
            'tags': [f'tag{j}' for j in range(i % 5)]
        }
        conversations.append(conversation)
    
    return conversations

# 测试分页和过滤
sample_data = create_sample_conversations()
filter_helper = DataFilter()
pagination_helper = PaginationHelper()

print(f"   🧪 测试分页和过滤功能:")
print(f"      📊 示例数据: {len(sample_data)}条对话记录")

# 测试过滤
filter_params = FilterParams(
    search="对话",
    status="active",
    category="technical"
)

filtered_data = filter_helper.filter_conversations(sample_data, filter_params)
print(f"      🔍 过滤结果: {len(filtered_data)}条记录")

# 测试排序
sort_params = SortParams(
    sort_by="created_at",
    sort_order="desc"
)

sorted_data = filter_helper.sort_data(filtered_data, sort_params)
print(f"      📊 排序完成: 按创建时间降序")

# 测试分页
pagination_params = PaginationParams(page=1, size=10)
paginated_data = pagination_helper.paginate_data(sorted_data, pagination_params.page, pagination_params.size)
print(f"      📄 分页结果: 第{pagination_params.page}页，每页{pagination_params.size}条，实际{len(paginated_data)}条")

## 6. 高级RESTful特性

### 6.1 错误响应标准化 (RFC 7807 Problem Details)

RFC 7807 Problem Details 提供了机器可读的错误响应格式，便于客户端程序化处理错误。

**核心优势：**
- 标准化错误格式
- 机器可读的错误类型
- 详细的错误上下文信息
- 支持字段级验证错误

**标准格式：**
```json
{
  "type": "https://api.example.com/problems/validation-error",
  "title": "Validation Error",
  "status": 422,
  "detail": "请求参数验证失败",
  "instance": "/api/conversations",
  "errors": {
    "title": ["不能为空"],
    "email": ["格式不正确"]
  },
  "timestamp": "2024-01-01T12:00:00Z",
  "trace_id": "550e8400-e29b-41d4-a716-446655440000"
}
```

In [ ]:
print(f"\n🚨 11. 高级RESTful特性:")

# RFC 7807 Problem Details 模型
class ProblemDetail(BaseModel):
    """RFC 7807 Problem Details 标准错误响应"""
    type: str = Field("about:blank", description="问题类型URI")
    title: str = Field(..., description="问题标题")
    status: int = Field(..., description="HTTP状态码")
    detail: str = Field(..., description="问题详细描述")
    instance: Optional[str] = Field(None, description="问题实例标识")
    errors: Optional[Dict[str, List[str]]] = Field(None, description="字段验证错误")
    timestamp: datetime = Field(default_factory=datetime.now, description="错误时间")
    trace_id: Optional[str] = Field(None, description="追踪ID")

# 错误响应构建器
class ErrorResponseBuilder:
    """标准化错误响应构建器"""
    
    problem_types = {
        "validation_error": "https://api.example.com/problems/validation-error",
        "not_found": "https://api.example.com/problems/not-found",
        "rate_limit_exceeded": "https://api.example.com/problems/rate-limit-exceeded",
        "authentication_error": "https://api.example.com/problems/authentication-error",
        "authorization_error": "https://api.example.com/problems/authorization-error",
        "internal_error": "https://api.example.com/problems/internal-error",
        "service_unavailable": "https://api.example.com/problems/service-unavailable"
    }
    
    @classmethod
    def create_problem_detail(
        cls,
        problem_type: str,
        status: int,
        detail: str,
        instance: Optional[str] = None,
        errors: Optional[Dict[str, List[str]]] = None,
        trace_id: Optional[str] = None
    ) -> ProblemDetail:
        """创建问题详情"""
        type_uri = cls.problem_types.get(problem_type, "about:blank")
        title = problem_type.replace('_', ' ').title()
        
        return ProblemDetail(
            type=type_uri,
            title=title,
            status=status,
            detail=detail,
            instance=instance,
            errors=errors,
            trace_id=trace_id or str(uuid.uuid4())
        )
    
    @classmethod
    def validation_error(cls, errors: Dict[str, List[str]], detail: str = "请求验证失败") -> ProblemDetail:
        """创建验证错误响应"""
        return cls.create_problem_detail(
            "validation_error",
            422,
            detail,
            errors=errors
        )
    
    @classmethod
    def not_found(cls, resource: str, resource_id: str) -> ProblemDetail:
        """创建资源未找到错误"""
        return cls.create_problem_detail(
            "not_found",
            404,
            f"{resource} '{resource_id}' 不存在",
            instance=f"/{resource}/{resource_id}"
        )
    
    @classmethod
    def rate_limit_exceeded(cls, retry_after: int) -> ProblemDetail:
        """创建速率限制错误"""
        return cls.create_problem_detail(
            "rate_limit_exceeded",
            429,
            f"请求频率超限，请在 {retry_after} 秒后重试",
            errors={"retry_after": [str(retry_after)]}
        )

# ETag 和条件请求支持
class ETagManager:
    """ETag管理器 - 支持乐观并发控制"""
    
    @staticmethod
    def generate_etag(data: Dict[str, Any]) -> str:
        """生成ETag"""
        data_str = json.dumps(data, sort_keys=True)
        return f'"{hashlib.md5(data_str.encode()).hexdigest()}"'
    
    @staticmethod
    def check_if_match(request: Request, current_etag: str) -> bool:
        """检查If-Match头部"""
        if_match = request.headers.get("If-Match")
        if if_match and if_match != current_etag:
            return False
        return True
    
    @staticmethod
    def check_if_none_match(request: Request, current_etag: str) -> bool:
        """检查If-None-Match头部"""
        if_none_match = request.headers.get("If-None-Match")
        if if_none_match and if_none_match == current_etag:
            return True
        return False

print(f"   ✅ RFC 7807 Problem Details 和 ETag 管理器创建完成")
print(f"      - 标准化错误响应格式")
print(f"      - 乐观并发控制支持")
print(f"      - 详细的错误上下文信息")

### 6.2 幂等性键和批量操作

**幂等性键 (Idempotency Keys):**
- 确保重复请求产生相同结果
- 防止网络重试导致的重复操作
- 对POST/PATCH等非幂等操作特别重要

**批量操作 (Bulk Operations):**
- 减少网络往返次数
- 提高大量数据操作效率
- 支持部分成功和错误回滚

**稀疏字段集 (Sparse Fieldsets):**
- 只返回客户端需要的字段
- 减少网络传输开销
- 提高API响应速度

In [ ]:
print(f"\n🔄 12. 幂等性键和批量操作:")

# 幂等性键管理器
class IdempotencyManager:
    """幂等性键管理器 - 防止重复操作"""
    
    def __init__(self):
        self.idempotency_store = {}  # {key: (response, timestamp)}
        self.default_ttl = 3600  # 1小时
    
    async def generate_key(self, request: Request) -> str:
        """生成幂等性键"""
        # 优先使用客户端提供的键
        client_key = request.headers.get("Idempotency-Key")
        if client_key:
            return client_key
        
        # 否则基于请求内容生成
        content = await request.body()
        key_data = f"{request.method}:{request.url}:{content}"
        return hashlib.sha256(key_data.encode()).hexdigest()
    
    def check_and_store(self, key: str, response: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        """检查并存储幂等性响应"""
        if key in self.idempotency_store:
            stored_response, timestamp = self.idempotency_store[key]
            if time.time() - timestamp < self.default_ttl:
                return stored_response
        
        # 存储新响应
        self.idempotency_store[key] = (response, time.time())
        return None

# 批量操作处理器
class BulkOperationHandler:
    """批量操作处理器"""
    
    @staticmethod
    def process_bulk_create(items: List[Dict[str, Any]], validator_func) -> Dict[str, Any]:
        """处理批量创建"""
        results = []
        errors = []
        
        for i, item in enumerate(items):
            try:
                # 验证单个项目
                validated_item = validator_func(item)
                # 模拟创建操作
                created_item = {**validated_item, "id": len(results) + 1}
                results.append(created_item)
            except Exception as e:
                errors.append({
                    "index": i,
                    "error": str(e),
                    "item": item
                })
        
        return {
            "successful": len(results),
            "failed": len(errors),
            "results": results,
            "errors": errors
        }
    
    @staticmethod
    def process_bulk_update(items: List[Dict[str, Any]], update_func) -> Dict[str, Any]:
        """处理批量更新"""
        results = []
        errors = []
        
        for i, item in enumerate(items):
            try:
                item_id = item.get("id")
                if not item_id:
                    raise ValueError("缺少ID字段")
                
                # 执行更新
                updated_item = update_func(item_id, item)
                results.append(updated_item)
            except Exception as e:
                errors.append({
                    "index": i,
                    "error": str(e),
                    "item": item
                })
        
        return {
            "successful": len(results),
            "failed": len(errors),
            "results": results,
            "errors": errors
        }

# 稀疏字段集处理器
class SparseFieldsetHandler:
    """稀疏字段集处理器 - 只返回请求的字段"""
    
    @staticmethod
    def filter_fields(data: Dict[str, Any], fields: Optional[List[str]]) -> Dict[str, Any]:
        """过滤字段"""
        if not fields:
            return data
        
        filtered_data = {}
        for field in fields:
            if field in data:
                filtered_data[field] = data[field]
        
        return filtered_data
    
    @staticmethod
    def parse_fields_param(fields_param: Optional[str]) -> Optional[List[str]]:
        """解析fields参数"""
        if not fields_param:
            return None
        
        return [field.strip() for field in fields_param.split(',') if field.strip()]

# 内容协商处理器
class ContentNegotiationHandler:
    """内容协商处理器"""
    
    supported_formats = {
        "application/json": "json",
        "application/xml": "xml",
        "text/csv": "csv"
    }
    
    @classmethod
    def negotiate_content_type(cls, accept_header: str) -> str:
        """协商内容类型"""
        if not accept_header:
            return "application/json"
        
        # 简单的内容协商实现
        accepted_types = [t.strip().split(';')[0] for t in accept_header.split(',')]
        
        for content_type in accepted_types:
            if content_type in cls.supported_formats:
                return content_type
        
        return "application/json"  # 默认返回JSON
    
    @classmethod
    def format_response(cls, data: Any, content_type: str) -> Any:
        """格式化响应"""
        format_type = cls.supported_formats.get(content_type, "json")
        
        if format_type == "json":
            return data
        elif format_type == "xml":
            # 简化的XML转换（实际应用中应使用专门的库）
            return {"xml_data": str(data)}
        elif format_type == "csv":
            # 简化的CSV转换（实际应用中应使用pandas等库）
            return {"csv_data": str(data)}
        
        return data

# 创建处理器实例
idempotency_manager = IdempotencyManager()
bulk_handler = BulkOperationHandler()
fieldset_handler = SparseFieldsetHandler()
content_handler = ContentNegotiationHandler()

print(f"   ✅ 高级功能处理器创建完成")
print(f"      - IdempotencyManager: 幂等性键管理")
print(f"      - BulkOperationHandler: 批量操作处理")
print(f"      - SparseFieldsetHandler: 稀疏字段集")
print(f"      - ContentNegotiationHandler: 内容协商")

### 6.3 OpenAPI文档生成和API弃用策略

**OpenAPI/Swagger 集成:**
- 自动生成API文档
- 交互式API测试界面
- 客户端SDK生成支持

**API弃用策略:**
- Sunset头部通知
- 版本迁移指导
- 平滑过渡机制

**请求压缩:**
- Gzip压缩支持
- 减少网络传输
- 提升响应性能

In [ ]:
print(f"\n📚 13. OpenAPI文档生成和API弃用策略:")

# API弃用管理器
class DeprecationManager:
    """API弃用管理器"""
    
    def __init__(self):
        self.deprecated_endpoints = {
            "/api/v1/conversations": {
                "deprecation_date": datetime(2024, 12, 1),
                "sunset_date": datetime(2025, 6, 1),
                "alternative": "/api/v2/conversations",
                "reason": "升级到更高效的分页机制"
            },
            "/api/v1/chat": {
                "deprecation_date": datetime(2024, 11, 1),
                "sunset_date": datetime(2025, 3, 1),
                "alternative": "/api/v2/chat",
                "reason": "支持流式响应和更好的错误处理"
            }
        }
    
    def is_deprecated(self, endpoint: str) -> bool:
        """检查端点是否已弃用"""
        return endpoint in self.deprecated_endpoints
    
    def get_deprecation_info(self, endpoint: str) -> Optional[Dict[str, Any]]:
        """获取弃用信息"""
        return self.deprecated_endpoints.get(endpoint)
    
    def add_deprecation_headers(self, endpoint: str, headers: Dict[str, str]) -> Dict[str, str]:
        """添加弃用头部"""
        if not self.is_deprecated(endpoint):
            return headers
        
        info = self.deprecated_endpoints[endpoint]
        headers.update({
            "Deprecation": "true",
            "Sunset": info["sunset_date"].strftime("%a, %d %b %Y %H:%M:%S GMT"),
            "Link": f'<{info["alternative"]}>; rel="successor-version"',
            "X-API-Deprecation-Reason": info["reason"]
        })
        
        return headers

# 请求压缩处理器
class CompressionHandler:
    """请求压缩处理器"""
    
    @staticmethod
    def should_compress(content_type: str, content_length: int) -> bool:
        """判断是否应该压缩"""
        compressible_types = [
            "application/json",
            "text/html",
            "text/css",
            "text/javascript",
            "application/xml"
        ]
        
        return (
            content_type in compressible_types and
            content_length > 1024  # 只压缩大于1KB的响应
        )
    
    @staticmethod
    def add_compression_headers(headers: Dict[str, str]) -> Dict[str, str]:
        """添加压缩相关头部"""
        headers.update({
            "Content-Encoding": "gzip",
            "Vary": "Accept-Encoding"
        })
        return headers

# OpenAPI文档增强器
class OpenAPIEnhancer:
    """OpenAPI文档增强器"""
    
    @staticmethod
    def enhance_openapi_spec(app: FastAPI) -> Dict[str, Any]:
        """增强OpenAPI规范"""
        openapi_schema = app.openapi()
        
        # 添加自定义信息
        openapi_schema["info"].update({
            "description": "LangChain RESTful API - 生产级实现",
            "termsOfService": "https://example.com/terms/",
            "contact": {
                "name": "API Support",
                "url": "https://example.com/support",
                "email": "api@example.com"
            },
            "license": {
                "name": "MIT",
                "url": "https://opensource.org/licenses/MIT"
            }
        })
        
        # 添加服务器配置
        openapi_schema["servers"] = [
            {
                "url": "http://localhost:8000",
                "description": "开发环境"
            },
            {
                "url": "https://api.example.com",
                "description": "生产环境"
            }
        ]
        
        # 添加安全配置
        openapi_schema["components"]["securitySchemes"] = {
            "ApiKeyAuth": {
                "type": "apiKey",
                "in": "header",
                "name": "X-API-Key"
            },
            "BearerAuth": {
                "type": "http",
                "scheme": "bearer",
                "bearerFormat": "JWT"
            }
        }
        
        # 添加标签分组
        openapi_schema["tags"] = [
            {
                "name": "conversations",
                "description": "对话管理相关操作"
            },
            {
                "name": "documents",
                "description": "文档管理相关操作"
            },
            {
                "name": "models",
                "description": "AI模型相关操作"
            },
            {
                "name": "users",
                "description": "用户管理相关操作"
            }
        ]
        
        return openapi_schema

# 创建增强器实例
deprecation_manager = DeprecationManager()
compression_handler = CompressionHandler()
openapi_enhancer = OpenAPIEnhancer()

print(f"   ✅ OpenAPI和弃用管理器创建完成")
print(f"      - DeprecationManager: API弃用策略管理")
print(f"      - CompressionHandler: 响应压缩处理")
print(f"      - OpenAPIEnhancer: API文档增强")

print(f"\n📊 弃用端点信息:")
for endpoint, info in deprecation_manager.deprecated_endpoints.items():
    print(f"   - {endpoint}: {info['reason']}")
    print(f"     替代方案: {info['alternative']}")
    print(f"     停用日期: {info['sunset_date'].strftime('%Y-%m-%d')}")

### 6.4 综合示例：生产级LangChain API端点

**集成多种高级特性的实际应用场景：**
- 聊天API：幂等性重试 + ETag缓存 + 错误标准化
- 文档批量上传：批量操作 + 压缩 + 内容协商
- 对话历史查询：稀疏字段 + 分页过滤 + HATEOAS

**LangChain特定优化策略：**
- 聊天重试机制防止重复消息
- 文档批量处理提升RAG效率
- 流式响应减少延迟感知

In [ ]:
print(f"\n🔗 14. 综合示例：生产级LangChain API端点:")

if FASTAPI_AVAILABLE:
    # 创建综合示例应用
    comprehensive_app = FastAPI(
        title="LangChain Production API",
        description="集成多种高级特性的生产级API",
        version="1.0.0"
    )
    
    # 初始化所有管理器
    idempotency_manager = IdempotencyManager()
    etag_manager = ETagManager()
    error_builder = ErrorResponseBuilder()
    deprecation_manager = DeprecationManager()
    fieldset_handler = SparseFieldsetHandler()
    bulk_handler = BulkOperationHandler()
    
    # 综合聊天API端点 - 集成多种高级特性
    @comprehensive_app.post("/api/v2/chat")
    async def comprehensive_chat_endpoint(
        request: Request,
        chat_data: Dict[str, Any],
        idempotency_key: Optional[str] = Header(None, alias="Idempotency-Key"),
        if_none_match: Optional[str] = Header(None, alias="If-None-Match"),
        accept: str = Header("application/json")
    ):
        """生产级聊天API - 集成所有高级特性"""
        
        # 1. 幂等性检查
        if idempotency_key:
            cached_response = idempotency_manager.check_and_store(idempotency_key, chat_data)
            if cached_response:
                return JSONResponse(
                    status_code=200,
                    content=cached_response,
                    headers={"X-Idempotent": "true"}
                )
        
        # 2. 请求验证
        if "message" not in chat_data:
            problem_detail = error_builder.validation_error(
                {"message": ["消息内容是必需的"]},
                "聊天请求验证失败"
            )
            return JSONResponse(
                status_code=422,
                content=problem_detail.dict(),
                headers={"Content-Type": "application/problem+json"}
            )
        
        # 3. 模拟AI处理
        await asyncio.sleep(0.5)  # 模拟处理时间
        response_data = {
            "id": str(uuid.uuid4()),
            "message": chat_data["message"],
            "response": f"AI回答: {chat_data['message']}",
            "model": chat_data.get("model", "gpt-3.5-turbo"),
            "timestamp": datetime.now().isoformat(),
            "tokens_used": 150,
            "processing_time": 0.5
        }
        
        # 4. ETag生成和条件请求
        etag = etag_manager.generate_etag(response_data)
        if etag_manager.check_if_none_match(request, etag):
            return Response(
                status_code=304,
                headers={"ETag": etag}
            )
        
        # 5. 内容协商
        content_type = content_handler.negotiate_content_type(accept)
        formatted_response = content_handler.format_response(response_data, content_type)
        
        # 6. 存储幂等性响应
        if idempotency_key:
            idempotency_manager.idempotency_store[idempotency_key] = (response_data, time.time())
        
        # 7. 构建最终响应
        headers = {
            "ETag": etag,
            "Content-Type": content_type,
            "Cache-Control": "private, max-age=300",
            "X-Processing-Time": "0.5s"
        }
        
        # 8. 添加压缩头部（如果需要）
        if compression_handler.should_compress(content_type, len(str(formatted_response))):
            headers = compression_handler.add_compression_headers(headers)
        
        return JSONResponse(
            content=formatted_response,
            headers=headers
        )
    
    # 批量文档上传端点
    @comprehensive_app.post("/api/v2/documents/bulk")
    async def bulk_upload_documents(
        request: Request,
        documents: List[Dict[str, Any]],
        fields: Optional[str] = Query(None, description="返回字段")
    ):
        """批量文档上传 - 集成批量操作和稀疏字段"""
        
        def validate_document(doc):
            """验证文档"""
            if "title" not in doc or not doc["title"].strip():
                raise ValueError("文档标题不能为空")
            if "content" not in doc or not doc["content"].strip():
                raise ValueError("文档内容不能为空")
            return doc
        
        def create_document(validated_doc):
            """创建文档"""
            return {
                **validated_doc,
                "id": str(uuid.uuid4()),
                "created_at": datetime.now().isoformat(),
                "status": "processed"
            }
        
        # 执行批量操作
        bulk_result = bulk_handler.process_bulk_create(documents, validate_document)
        
        # 处理创建结果
        created_docs = []
        for doc in bulk_result["results"]:
            created_doc = create_document(doc)
            created_docs.append(created_doc)
        
        # 稀疏字段过滤
        requested_fields = fieldset_handler.parse_fields_param(fields)
        if requested_fields:
            created_docs = [
                fieldset_handler.filter_fields(doc, requested_fields)
                for doc in created_docs
            ]
        
        # 构建响应
        response_data = {
            "batch_id": str(uuid.uuid4()),
            "summary": {
                "total_requested": len(documents),
                "successful": bulk_result["successful"],
                "failed": bulk_result["failed"]
            },
            "documents": created_docs,
            "errors": bulk_result["errors"]
        }
        
        return JSONResponse(
            status_code=207,  # Multi-Status
            content=response_data,
            headers={
                "Content-Type": "application/json",
                "X-Batch-Operation": "true"
            }
        )
    
    # 增强的对话历史查询端点
    @comprehensive_app.get("/api/v2/conversations/{conversation_id}/history")
    async def get_conversation_history(
        conversation_id: str,
        page: int = Query(1, ge=1),
        size: int = Query(20, ge=1, le=100),
        fields: Optional[str] = Query(None),
        include_metadata: bool = Query(False)
    ):
        """增强的对话历史查询 - 集成分页、稀疏字段、HATEOAS"""
        
        # 模拟对话历史数据
        history_data = [
            {
                "id": i,
                "conversation_id": conversation_id,
                "role": "user" if i % 2 == 0 else "assistant",
                "content": f"消息内容 {i}",
                "timestamp": (datetime.now() - timedelta(minutes=i*5)).isoformat(),
                "metadata": {
                    "tokens": 50,
                    "model": "gpt-3.5-turbo",
                    "processing_time": 0.3
                } if include_metadata else None
            }
            for i in range(1, 101)  # 100条消息
        ]
        
        # 分页处理
        paginated_data = PaginationHelper.paginate_data(history_data, page, size)
        
        # 稀疏字段过滤
        requested_fields = fieldset_handler.parse_fields_param(fields)
        if requested_fields:
            paginated_data = [
                fieldset_handler.filter_fields(msg, requested_fields)
                for msg in paginated_data
            ]
        
        # 创建分页元数据
        pagination_meta = PaginationHelper.create_pagination_meta(page, size, len(history_data))
        
        # 创建HATEOAS链接
        hateoas_links = hateoas_builder.create_collection_links(
            f"/api/v2/conversations/{conversation_id}/history",
            page, size, len(history_data)
        )
        
        # 添加操作链接
        hateoas_links.update({
            "conversation": hateoas_builder.create_link(
                f"/api/v2/conversations/{conversation_id}",
                "conversation",
                title="对话详情"
            ),
            "export": hateoas_builder.create_link(
                f"/api/v2/conversations/{conversation_id}/export",
                "export",
                method="POST",
                title="导出对话"
            )
        })
        
        return {
            "data": paginated_data,
            "meta": pagination_meta.dict(),
            "_links": {rel: link.dict() for rel, link in hateoas_links.items()}
        }
    
    print(f"   ✅ 综合示例API端点创建完成")
    print(f"      - POST /api/v2/chat: 集成幂等性、ETag、内容协商")
    print(f"      - POST /api/v2/documents/bulk: 批量操作和稀疏字段")
    print(f"      - GET /api/v2/conversations/{{id}}/history: 增强分页查询")
    
    print(f"\n🎯 LangChain特定特性映射:")
    langchain_features = {
        "聊天重试机制": "幂等性键防止网络重试导致重复消息",
        "文档批量处理": "批量操作提升RAG文档导入效率",
        "流式响应缓存": "ETag支持聊天响应缓存和条件请求",
        "对话历史优化": "稀疏字段减少大历史记录传输开销",
        "多格式导出": "内容协商支持JSON/XML/CSV格式导出",
        "API版本演进": "弃用策略确保平滑升级路径"
    }
    
    for feature, description in langchain_features.items():
        print(f"   - {feature}: {description}")
        
else:
    print(f"   ❌ FastAPI不可用，跳过综合示例")

## 3. API版本控制

### 3.1 版本控制策略

In [ ]:
print(f"\n🏷️ 5. API版本控制:")

# 版本控制策略枚举
class VersioningStrategy:
    """版本控制策略"""
    
    URL_PATH = "url_path"  # URL路径版本控制
    HEADER = "header"      # 请求头版本控制
    QUERY_PARAM = "query"  # 查询参数版本控制
    SUBDOMAIN = "subdomain" # 子域名版本控制
    
    strategies = {
        URL_PATH: {
            "description": "在URL路径中包含版本号",
            "example": "/api/v1/users, /api/v2/users",
            "pros": ["清晰明确", "易于缓存", "SEO友好"],
            "cons": ["URL变更", "需要路由重写"]
        },
        HEADER: {
            "description": "使用请求头指定版本",
            "example": "Accept: application/vnd.api+json;version=1",
            "pros": ["URL不变", "灵活配置"],
            "cons": ["不易调试", "缓存困难"]
        },
        QUERY_PARAM: {
            "description": "使用查询参数指定版本",
            "example": "/api/users?version=1",
            "pros": ["简单实现", "易于测试"],
            "cons": ["不够RESTful", "参数污染"]
        },
        SUBDOMAIN: {
            "description": "使用子域名区分版本",
            "example": "v1.api.example.com, v2.api.example.com",
            "pros": ["完全隔离", "易于部署"],
            "cons": ["配置复杂", "SSL证书问题"]
        }
    }
    
    @classmethod
    def print_strategies(cls):
        """打印版本控制策略"""
        print(f"   📋 API版本控制策略:")
        for strategy, info in cls.strategies.items():
            print(f"\n      🔹 {strategy.replace('_', ' ').title()}:")
            print(f"         描述: {info['description']}")
            print(f"         示例: {info['example']}")
            print(f"         优点: {', '.join(info['pros'])}")
            print(f"         缺点: {', '.join(info['cons'])}")

# 版本信息模型
class APIVersion(BaseModel):
    """API版本信息"""
    version: str = Field(..., description="版本号")
    status: str = Field(..., description="版本状态")
    release_date: datetime = Field(..., description="发布日期")
    deprecation_date: Optional[datetime] = Field(None, description="弃用日期")
    sunset_date: Optional[datetime] = Field(None, description="停用日期")
    features: List[str] = Field(default_factory=list, description="新功能列表")
    breaking_changes: List[str] = Field(default_factory=list, description="破坏性变更")

# 版本管理器
class VersionManager:
    """API版本管理器"""
    
    def __init__(self):
        self.versions = {
            "v1": APIVersion(
                version="v1",
                status="stable",
                release_date=datetime(2024, 1, 1),
                features=["基础聊天API", "文档检索", "用户管理"]
            ),
            "v2": APIVersion(
                version="v2",
                status="beta",
                release_date=datetime(2024, 6, 1),
                features=["流式响应", "批量处理", "高级过滤"],
                breaking_changes=["响应格式变更", "认证方式更新"]
            ),
            "v3": APIVersion(
                version="v3",
                status="alpha",
                release_date=datetime(2024, 12, 1),
                features=["WebSocket支持", "多模态处理", "工作流引擎"]
            )
        }
        self.default_version = "v1"
    
    def get_version(self, version: str) -> Optional[APIVersion]:
        """获取版本信息"""
        return self.versions.get(version)
    
    def is_version_supported(self, version: str) -> bool:
        """检查版本是否支持"""
        return version in self.versions
    
    def get_latest_version(self) -> str:
        """获取最新稳定版本"""
        stable_versions = [
            v for v in self.versions.values() 
            if v.status == "stable"
        ]
        if stable_versions:
            return max(stable_versions, key=lambda x: x.release_date).version
        return self.default_version
    
    def validate_version(self, version: str) -> str:
        """验证并返回有效版本"""
        if not version:
            return self.default_version
        
        if self.is_version_supported(version):
            return version
        
        # 如果版本不支持，返回最新稳定版本
        return self.get_latest_version()

# 打印版本控制策略
VersioningStrategy.print_strategies()

# 创建版本管理器
version_manager = VersionManager()

print(f"\n   📊 版本管理器测试:")
for version in ["v1", "v2", "v3", "v4"]:
    is_supported = version_manager.is_version_supported(version)
    valid_version = version_manager.validate_version(version)
    print(f"      {version:<4} : {'✅' if is_supported else '❌'} -> {valid_version}")

print(f"\n   🎯 最新稳定版本: {version_manager.get_latest_version()}")

### 3.2 版本控制中间件实现

In [ ]:
print(f"\n🔧 6. 版本控制中间件实现:")

if FASTAPI_AVAILABLE:
    # 版本控制中间件
    class VersioningMiddleware:
        """版本控制中间件"""
        
        def __init__(self, version_manager: VersionManager, strategy: str = VersioningStrategy.URL_PATH):
            self.version_manager = version_manager
            self.strategy = strategy
        
        async def extract_version_from_path(self, request: Request) -> str:
            """从URL路径提取版本"""
            path_parts = request.url.path.strip('/').split('/')
            if len(path_parts) >= 2 and path_parts[0] == 'api':
                potential_version = path_parts[1]
                if potential_version.startswith('v'):
                    return potential_version
            return None
        
        async def extract_version_from_header(self, request: Request) -> str:
            """从请求头提取版本"""
            accept_header = request.headers.get('Accept', '')
            if 'version=' in accept_header:
                version_part = accept_header.split('version=')[1].split(';')[0].strip()
                return version_part
            
            api_version_header = request.headers.get('API-Version', '')
            if api_version_header:
                return api_version_header
            
            return None
        
        async def extract_version_from_query(self, request: Request) -> str:
            """从查询参数提取版本"""
            query_params = dict(request.query_params)
            return query_params.get('version')
        
        async def get_version(self, request: Request) -> str:
            """获取请求版本"""
            version = None
            
            if self.strategy == VersioningStrategy.URL_PATH:
                version = await self.extract_version_from_path(request)
            elif self.strategy == VersioningStrategy.HEADER:
                version = await self.extract_version_from_header(request)
            elif self.strategy == VersioningStrategy.QUERY_PARAM:
                version = await self.extract_version_from_query(request)
            
            return self.version_manager.validate_version(version)
        
        def create_middleware(self):
            """创建FastAPI中间件"""
            async def middleware(request: Request, call_next):
                # 提取版本
                version = await self.get_version(request)
                
                # 将版本信息添加到请求状态
                request.state.api_version = version
                request.state.version_info = self.version_manager.get_version(version)
                
                # 处理请求
                response = await call_next(request)
                
                # 添加版本信息到响应头
                response.headers['API-Version'] = version
                
                return response
            
            return middleware
    
    # 创建带版本控制的FastAPI应用
    versioned_app = FastAPI(
        title="LangChain Versioned API",
        description="支持版本控制的LangChain API",
        version="1.0.0"
    )
    
    # 初始化版本管理器和中间件
    version_manager = VersionManager()
    versioning_middleware = VersioningMiddleware(version_manager)
    
    # 添加版本控制中间件
    versioned_app.middleware("http")(versioning_middleware.create_middleware())
    
    # 版本信息端点
    @versioned_app.get("/api/version")
    async def get_api_version(request: Request):
        """获取API版本信息"""
        version = request.state.api_version
        version_info = request.state.version_info
        
        return {
            "current_version": version,
            "version_info": version_info.dict() if version_info else None,
            "supported_versions": list(version_manager.versions.keys()),
            "default_version": version_manager.default_version
        }
    
    # 版本特定的聊天端点
    @versioned_app.get("/api/v1/conversations")
    async def get_conversations_v1(request: Request):
        """v1版本的对话列表"""
        return {
            "version": "v1",
            "conversations": [
                {"id": 1, "title": "对话1", "status": "active"},
                {"id": 2, "title": "对话2", "status": "archived"}
            ]
        }
    
    @versioned_app.get("/api/v2/conversations")
    async def get_conversations_v2(request: Request):
        """v2版本的对话列表（增强功能）"""
        return {
            "version": "v2",
            "conversations": [
                {
                    "id": 1,
                    "title": "对话1",
                    "status": "active",
                    "metadata": {"created_at": "2024-01-01", "message_count": 10},
                    "features": ["streaming", "batch_processing"]
                },
                {
                    "id": 2,
                    "title": "对话2",
                    "status": "archived",
                    "metadata": {"created_at": "2024-01-02", "message_count": 5},
                    "features": ["basic"]
                }
            ],
            "pagination": {
                "page": 1,
                "size": 10,
                "total": 2
            }
        }
    
    print(f"   ✅ 版本控制中间件实现完成")
    print(f"      - 支持URL路径版本控制")
    print(f"      - 支持请求头版本控制")
    print(f"      - 支持查询参数版本控制")
    print(f"      - 自动版本验证和回退")
    print(f"      - 版本信息响应头")
    
    print(f"\n   🌐 版本化端点:")
    print(f"      - GET /api/version: 获取版本信息")
    print(f"      - GET /api/v1/conversations: v1对话列表")
    print(f"      - GET /api/v2/conversations: v2对话列表（增强版）")
else:
    print(f"   ❌ FastAPI不可用，跳过版本控制中间件实现")

## 4. 速率限制和缓存

### 4.1 速率限制实现

In [ ]:
print(f"\n🚦 7. 速率限制和缓存:")

# 简单的内存速率限制器
class MemoryRateLimiter:
    """内存速率限制器"""
    
    def __init__(self):
        self.requests = {}  # {client_ip: [(timestamp, count)]}
        self.limits = {
            "default": (100, 60),  # 100 requests per minute
            "chat": (20, 60),      # 20 chat requests per minute
            "upload": (5, 60),     # 5 uploads per minute
        }
    
    def is_allowed(self, client_id: str, limit_type: str = "default") -> Tuple[bool, Dict[str, Any]]:
        """检查是否允许请求"""
        now = time.time()
        max_requests, window_seconds = self.limits.get(limit_type, self.limits["default"])
        
        # 清理过期记录
        if client_id in self.requests:
            self.requests[client_id] = [
                (timestamp, count) for timestamp, count in self.requests[client_id]
                if now - timestamp < window_seconds
            ]
        else:
            self.requests[client_id] = []
        
        # 计算当前窗口内的请求数
        current_requests = sum(count for _, count in self.requests[client_id])
        
        if current_requests >= max_requests:
            # 计算重置时间
            oldest_request = min(self.requests[client_id], key=lambda x: x[0], default=(now, 0))
            reset_time = oldest_request[0] + window_seconds
            
            return False, {
                "allowed": False,
                "limit": max_requests,
                "remaining": 0,
                "reset_time": int(reset_time),
                "retry_after": int(reset_time - now)
            }
        
        # 记录新请求
        self.requests[client_id].append((now, 1))
        
        remaining = max_requests - current_requests - 1
        reset_time = now + window_seconds
        
        return True, {
            "allowed": True,
            "limit": max_requests,
            "remaining": remaining,
            "reset_time": int(reset_time),
            "retry_after": 0
        }
    
    def set_limit(self, limit_type: str, max_requests: int, window_seconds: int):
        """设置限制规则"""
        self.limits[limit_type] = (max_requests, window_seconds)

# 缓存管理器
class CacheManager:
    """内存缓存管理器"""
    
    def __init__(self):
        self.cache = {}  # {key: (value, expiry_time)}
        self.default_ttl = 300  # 5 minutes
    
    def _generate_key(self, prefix: str, **kwargs) -> str:
        """生成缓存键"""
        key_data = sorted(kwargs.items())
        key_str = f"{prefix}:{hashlib.md5(str(key_data).encode()).hexdigest()}"
        return key_str
    
    def get(self, key: str) -> Optional[Any]:
        """获取缓存值"""
        if key in self.cache:
            value, expiry_time = self.cache[key]
            if time.time() < expiry_time:
                return value
            else:
                del self.cache[key]
        return None
    
    def set(self, key: str, value: Any, ttl: Optional[int] = None) -> None:
        """设置缓存值"""
        if ttl is None:
            ttl = self.default_ttl
        
        expiry_time = time.time() + ttl
        self.cache[key] = (value, expiry_time)
    
    def delete(self, key: str) -> bool:
        """删除缓存"""
        if key in self.cache:
            del self.cache[key]
            return True
        return False
    
    def clear_expired(self) -> int:
        """清理过期缓存"""
        now = time.time()
        expired_keys = [
            key for key, (_, expiry_time) in self.cache.items()
            if now >= expiry_time
        ]
        
        for key in expired_keys:
            del self.cache[key]
        
        return len(expired_keys)
    
    def get_cache_key_for_query(self, query_type: str, **params) -> str:
        """为查询生成缓存键"""
        return self._generate_key(f"query:{query_type}", **params)
    
    def get_cache_key_for_user(self, user_id: str, resource: str, **params) -> str:
        """为用户资源生成缓存键"""
        return self._generate_key(f"user:{user_id}:{resource}", **params)

# 创建速率限制器和缓存管理器
rate_limiter = MemoryRateLimiter()
cache_manager = CacheManager()

print(f"   ✅ 速率限制和缓存管理器创建完成")
print(f"      - MemoryRateLimiter: 基于内存的速率限制")
print(f"      - CacheManager: TTL缓存管理")
print(f"      - 支持不同类型的限制规则")
print(f"      - 自动过期缓存清理")

### 4.2 FastAPI集成速率限制和缓存

In [ ]:
print(f"\n🔗 8. FastAPI集成速率限制和缓存:")

if FASTAPI_AVAILABLE:
    # 速率限制中间件
    class RateLimitMiddleware:
        """速率限制中间件"""
        
        def __init__(self, rate_limiter: MemoryRateLimiter):
            self.rate_limiter = rate_limiter
        
        def create_middleware(self):
            """创建FastAPI中间件"""
            async def middleware(request: Request, call_next):
                # 获取客户端ID（IP地址或其他标识）
                client_id = get_remote_address(request)
                
                # 根据路径确定限制类型
                path = request.url.path
                if "/chat" in path:
                    limit_type = "chat"
                elif "/upload" in path:
                    limit_type = "upload"
                else:
                    limit_type = "default"
                
                # 检查速率限制
                allowed, limit_info = self.rate_limiter.is_allowed(client_id, limit_type)
                
                if not allowed:
                    return JSONResponse(
                        status_code=429,
                        content={
                            "error": "Rate limit exceeded",
                            "message": f"Rate limit exceeded. Try again in {limit_info['retry_after']} seconds.",
                            "limit_info": limit_info
                        },
                        headers={
                            "X-RateLimit-Limit": str(limit_info["limit"]),
                            "X-RateLimit-Remaining": str(limit_info["remaining"]),
                            "X-RateLimit-Reset": str(limit_info["reset_time"]),
                            "Retry-After": str(limit_info["retry_after"])
                        }
                    )
                
                # 处理请求
                response = await call_next(request)
                
                # 添加速率限制信息到响应头
                response.headers["X-RateLimit-Limit"] = str(limit_info["limit"])
                response.headers["X-RateLimit-Remaining"] = str(limit_info["remaining"])
                response.headers["X-RateLimit-Reset"] = str(limit_info["reset_time"])
                
                return response
            
            return middleware
    
    # 缓存装饰器
    def cache_response(ttl: int = 300, key_prefix: str = "api"):
        """缓存响应装饰器"""
        def decorator(func):
            async def wrapper(*args, **kwargs):
                # 生成缓存键
                cache_key = cache_manager.get_cache_key_for_query(
                    key_prefix,
                    args=str(args),
                    kwargs=str(kwargs)
                )
                
                # 尝试从缓存获取
                cached_result = cache_manager.get(cache_key)
                if cached_result is not None:
                    return cached_result
                
                # 执行函数
                result = await func(*args, **kwargs)
                
                # 缓存结果
                cache_manager.set(cache_key, result, ttl)
                
                return result
            
            return wrapper
        return decorator
    
    # 创建带速率限制和缓存的FastAPI应用
    cached_app = FastAPI(
        title="LangChain Cached API",
        description="支持速率限制和缓存的LangChain API",
        version="1.0.0"
    )
    
    # 添加速率限制中间件
    rate_limit_middleware = RateLimitMiddleware(rate_limiter)
    cached_app.middleware("http")(rate_limit_middleware.create_middleware())
    
    # 模拟LangChain服务
    class LangChainCachedService:
        """带缓存的LangChain服务"""
        
        @staticmethod
        @cache_response(ttl=600, key_prefix="chat")  # 10分钟缓存
        async def get_chat_response(message: str, model: str = "gpt-3.5-turbo") -> Dict[str, Any]:
            """获取聊天响应（带缓存）"""
            # 模拟AI处理时间
            await asyncio.sleep(0.5)
            
            return {
                "message": message,
                "response": f"AI对'{message}'的回答",
                "model": model,
                "timestamp": datetime.now().isoformat(),
                "cached": False
            }
        
        @staticmethod
        @cache_response(ttl=1800, key_prefix="rag")  # 30分钟缓存
        async def search_documents(query: str, top_k: int = 5) -> Dict[str, Any]:
            """文档搜索（带缓存）"""
            # 模拟搜索时间
            await asyncio.sleep(0.3)
            
            documents = [
                {"id": i, "content": f"关于{query}的文档{i}", "score": 0.9 - i * 0.1}
                for i in range(min(top_k, 5))
            ]
            
            return {
                "query": query,
                "documents": documents,
                "total": len(documents),
                "cached": False
            }
    
    # 创建服务实例
    langchain_service = LangChainCachedService()
    
    # API端点
    @cached_app.post("/api/chat")
    async def chat_endpoint(request: Dict[str, Any]):
        """聊天端点（带速率限制和缓存）"""
        message = request.get("message", "")
        model = request.get("model", "gpt-3.5-turbo")
        
        if not message:
            raise HTTPException(status_code=400, detail="Message is required")
        
        response = await langchain_service.get_chat_response(message, model)
        
        return JSONResponse(
            content=response,
            headers={
                "Cache-Control": "public, max-age=600",
                "X-Cache": "HIT" if response.get("cached") else "MISS"
            }
        )
    
    @cached_app.post("/api/search")
    async def search_endpoint(request: Dict[str, Any]):
        """文档搜索端点（带速率限制和缓存）"""
        query = request.get("query", "")
        top_k = request.get("top_k", 5)
        
        if not query:
            raise HTTPException(status_code=400, detail="Query is required")
        
        response = await langchain_service.search_documents(query, top_k)
        
        return JSONResponse(
            content=response,
            headers={
                "Cache-Control": "public, max-age=1800",
                "X-Cache": "HIT" if response.get("cached") else "MISS"
            }
        )
    
    @cached_app.get("/api/cache/stats")
    async def cache_stats():
        """缓存统计信息"""
        expired_count = cache_manager.clear_expired()
        
        return {
            "cache_size": len(cache_manager.cache),
            "expired_cleaned": expired_count,
            "default_ttl": cache_manager.default_ttl,
            "rate_limits": rate_limiter.limits
        }
    
    print(f"   ✅ FastAPI速率限制和缓存集成完成")
    print(f"      - 速率限制中间件: 自动限制请求频率")
    print(f"      - 缓存装饰器: 自动缓存API响应")
    print(f"      - 缓存控制头部: 标准HTTP缓存控制")
    print(f"      - 限制信息头部: 详细的速率限制信息")
    
    print(f"\n   🌐 缓存和限流端点:")
    print(f"      - POST /api/chat: 聊天API（20次/分钟，缓存10分钟）")
    print(f"      - POST /api/search: 搜索API（100次/分钟，缓存30分钟）")
    print(f"      - GET /api/cache/stats: 缓存统计信息")
else:
    print(f"   ❌ FastAPI不可用，跳过速率限制和缓存集成")

## 5. HATEOAS和超媒体导航

### 5.1 HATEOAS原则实现

In [ ]:
print(f"\n🔗 9. HATEOAS和超媒体导航:")

# HATEOAS链接模型
class Link(BaseModel):
    """HATEOAS链接模型"""
    href: str = Field(..., description="链接URL")
    rel: str = Field(..., description="链接关系")
    method: str = Field("GET", description="HTTP方法")
    title: Optional[str] = Field(None, description="链接标题")
    type: Optional[str] = Field(None, description="返回类型")
    templated: bool = Field(False, description="是否为模板链接")

# HATEOAS响应基类
class HATEOASResponse(BaseModel):
    """HATEOAS响应基类"""
    _links: Dict[str, Link] = Field(default_factory=dict, alias="_links", description="超媒体链接")
    _embedded: Optional[Dict[str, Any]] = Field(None, alias="_embedded", description="嵌入资源")
    
    def add_link(self, rel: str, link: Link):
        """添加链接"""
        self._links[rel] = link
    
    def add_links(self, links: Dict[str, Link]):
        """批量添加链接"""
        self._links.update(links)

# HATEOAS构建器
class HATEOASBuilder:
    """HATEOAS响应构建器"""
    
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url.rstrip('/')
    
    def create_link(self, href: str, rel: str, method: str = "GET", **kwargs) -> Link:
        """创建链接"""
        full_href = f"{self.base_url}{href}"
        return Link(href=full_href, rel=rel, method=method, **kwargs)
    
    def create_collection_links(self, resource_path: str, page: int = 1, size: int = 20, total: int = 0) -> Dict[str, Link]:
        """创建集合资源的导航链接"""
        links = {}
        
        # 自链接
        links["self"] = self.create_link(
            f"{resource_path}?page={page}&size={size}",
            "self"
        )
        
        # 第一页
        if page > 1:
            links["first"] = self.create_link(
                f"{resource_path}?page=1&size={size}",
                "first"
            )
        
        # 上一页
        if page > 1:
            links["prev"] = self.create_link(
                f"{resource_path}?page={page-1}&size={size}",
                "prev"
            )
        
        # 下一页
        pages = math.ceil(total / size) if size > 0 else 0
        if page < pages:
            links["next"] = self.create_link(
                f"{resource_path}?page={page+1}&size={size}",
                "next"
            )
        
        # 最后一页
        if page < pages:
            links["last"] = self.create_link(
                f"{resource_path}?page={pages}&size={size}",
                "last"
            )
        
        return links
    
    def create_resource_links(self, resource_path: str, resource_id: Union[str, int]) -> Dict[str, Link]:
        """创建单个资源的链接"""
        links = {}
        
        # 自链接
        links["self"] = self.create_link(
            f"{resource_path}/{resource_id}",
            "self"
        )
        
        # 集合链接
        collection_path = resource_path.rsplit('/', 1)[0]
        links["collection"] = self.create_link(
            collection_path,
            "collection"
        )
        
        # 更新链接
        links["update"] = self.create_link(
            f"{resource_path}/{resource_id}",
            "update",
            method="PUT"
        )
        
        # 删除链接
        links["delete"] = self.create_link(
            f"{resource_path}/{resource_id}",
            "delete",
            method="DELETE"
        )
        
        return links
    
    def create_action_links(self, resource_path: str, resource_id: Union[str, int]) -> Dict[str, Link]:
        """创建操作链接"""
        links = {}
        
        # 对话相关操作
        if "conversations" in resource_path:
            links["messages"] = self.create_link(
                f"{resource_path}/{resource_id}/messages",
                "messages",
                title="获取对话消息"
            )
            links["chat"] = self.create_link(
                f"{resource_path}/{resource_id}/chat",
                "chat",
                method="POST",
                title="发送消息"
            )
        
        # 文档相关操作
        elif "documents" in resource_path:
            links["download"] = self.create_link(
                f"{resource_path}/{resource_id}/download",
                "download",
                title="下载文档"
            )
            links["search"] = self.create_link(
                f"{resource_path}/search",
                "search",
                method="POST",
                title="搜索文档"
            )
        
        return links

# 创建HATEOAS构建器
hateoas_builder = HATEOASBuilder()

print(f"   ✅ HATEOAS模型和构建器创建完成")
print(f"      - Link: 超媒体链接模型")
print(f"      - HATEOASResponse: HATEOAS响应基类")
print(f"      - HATEOASBuilder: 链接构建工具")
print(f"      - 支持集合导航和资源操作链接")

### 5.2 完整的RESTful API示例

In [ ]:
print(f"\n🌐 10. 完整的RESTful API示例:")

if FASTAPI_AVAILABLE:
    # 创建完整的RESTful API应用
    restful_app = FastAPI(
        title="LangChain RESTful API",
        description="完整的RESTful API实现示例",
        version="1.0.0"
    )
    
    # 添加CORS中间件
    restful_app.add_middleware(
        CORSMiddleware,
        allow_origins=["*"],
        allow_credentials=True,
        allow_methods=["*"],
        allow_headers=["*"],
    )
    
    # 添加速率限制中间件
    rate_limit_middleware = RateLimitMiddleware(rate_limiter)
    restful_app.middleware("http")(rate_limit_middleware.create_middleware())
    
    # 模拟数据存储
    conversations_data = create_sample_conversations(100)
    
    # API根路径 - 发现API
    @restful_app.get("/api")
    async def api_root():
        """API根路径 - 发现可用资源"""
        links = {
            "conversations": hateoas_builder.create_link(
                "/api/conversations", "conversations", title="对话管理"
            ),
            "documents": hateoas_builder.create_link(
                "/api/documents", "documents", title="文档管理"
            ),
            "models": hateoas_builder.create_link(
                "/api/models", "models", title="模型管理"
            ),
            "users": hateoas_builder.create_link(
                "/api/users", "users", title="用户管理"
            ),
            "version": hateoas_builder.create_link(
                "/api/version", "version", title="版本信息"
            )
        }
        
        return {
            "message": "LangChain RESTful API",
            "version": "1.0.0",
            "_links": {rel: link.dict() for rel, link in links.items()}
        }
    
    # 对话列表 - 支持分页、过滤、排序
    @restful_app.get("/api/conversations")
    async def list_conversations(
        page: int = Query(1, ge=1, description="页码"),
        size: int = Query(20, ge=1, le=100, description="每页大小"),
        search: Optional[str] = Query(None, description="搜索关键词"),
        status: Optional[str] = Query(None, description="状态过滤"),
        sort_by: str = Query("created_at", description="排序字段"),
        sort_order: str = Query("desc", regex="^(asc|desc)$", description="排序方向")
    ):
        """获取对话列表（完整RESTful实现）"""
        # 构建查询参数
        filter_params = FilterParams(search=search, status=status)
        sort_params = SortParams(sort_by=sort_by, sort_order=sort_order)
        pagination_params = PaginationParams(page=page, size=size)
        
        # 过滤数据
        filtered_data = DataFilter.filter_conversations(conversations_data, filter_params)
        
        # 排序数据
        sorted_data = DataFilter.sort_data(filtered_data, sort_params)
        
        # 分页数据
        paginated_data = PaginationHelper.paginate_data(
            sorted_data, pagination_params.page, pagination_params.size
        )
        
        # 创建分页元数据
        pagination_meta = PaginationHelper.create_pagination_meta(
            page, size, len(filtered_data)
        )
        
        # 创建导航链接
        navigation_links = PaginationHelper.create_navigation_links(
            "/api/conversations", page, size, len(filtered_data)
        )
        
        # 创建HATEOAS链接
        hateoas_links = hateoas_builder.create_collection_links(
            "/api/conversations", page, size, len(filtered_data)
        )
        
        # 添加操作链接
        hateoas_links["create"] = hateoas_builder.create_link(
            "/api/conversations", "create", method="POST", title="创建对话"
        )
        
        return {
            "data": paginated_data,
            "meta": pagination_meta.dict(),
            "_links": {rel: link.dict() for rel, link in hateoas_links.items()}
        }
    
    # 单个对话详情
    @restful_app.get("/api/conversations/{conversation_id}")
    async def get_conversation(conversation_id: int):
        """获取对话详情"""
        # 查找对话
        conversation = next(
            (c for c in conversations_data if c["id"] == conversation_id),
            None
        )
        
        if not conversation:
            raise HTTPException(status_code=404, detail="对话不存在")
        
        # 创建HATEOAS链接
        hateoas_links = hateoas_builder.create_resource_links(
            "/api/conversations", conversation_id
        )
        
        # 添加操作链接
        action_links = hateoas_builder.create_action_links(
            "/api/conversations", conversation_id
        )
        hateoas_links.update(action_links)
        
        return {
            "data": conversation,
            "_links": {rel: link.dict() for rel, link in hateoas_links.items()}
        }
    
    # 创建对话
    @restful_app.post("/api/conversations")
    async def create_conversation(conversation_data: Dict[str, Any]):
        """创建对话"""
        # 验证数据
        if "title" not in conversation_data:
            raise HTTPException(status_code=400, detail="标题是必需的")
        
        # 创建新对话
        new_conversation = {
            "id": len(conversations_data) + 1,
            "title": conversation_data["title"],
            "description": conversation_data.get("description", ""),
            "status": "active",
            "category": conversation_data.get("category", "general"),
            "created_at": datetime.now().isoformat(),
            "message_count": 0,
            "tags": conversation_data.get("tags", [])
        }
        
        conversations_data.append(new_conversation)
        
        # 创建HATEOAS链接
        hateoas_links = hateoas_builder.create_resource_links(
            "/api/conversations", new_conversation["id"]
        )
        
        return JSONResponse(
            status_code=201,
            content={
                "data": new_conversation,
                "message": "对话创建成功",
                "_links": {rel: link.dict() for rel, link in hateoas_links.items()}
            }
        )
    
    # API统计信息
    @restful_app.get("/api/stats")
    async def get_api_stats():
        """获取API统计信息"""
        stats = {
            "conversations": {
                "total": len(conversations_data),
                "active": len([c for c in conversations_data if c["status"] == "active"]),
                "archived": len([c for c in conversations_data if c["status"] == "archived"]),
                "pending": len([c for c in conversations_data if c["status"] == "pending"])
            },
            "cache": {
                "size": len(cache_manager.cache),
                "default_ttl": cache_manager.default_ttl
            },
            "rate_limits": rate_limiter.limits,
            "version": "1.0.0",
            "timestamp": datetime.now().isoformat()
        }
        
        # 创建相关链接
        links = {
            "self": hateoas_builder.create_link("/api/stats", "self"),
            "conversations": hateoas_builder.create_link("/api/conversations", "conversations"),
            "cache_stats": hateoas_builder.create_link("/api/cache/stats", "cache_stats")
        }
        
        return {
            "data": stats,
            "_links": {rel: link.dict() for rel, link in links.items()}
        }
    
    print(f"   ✅ 完整RESTful API实现完成")
    print(f"      - API发现: /api 根路径")
    print(f"      - 对话管理: CRUD操作 + HATEOAS导航")
    print(f"      - 分页过滤: 完整的查询参数支持")
    print(f"      - 速率限制: 自动请求频率控制")
    print(f"      - 缓存支持: 提升API性能")
    print(f"      - 超媒体导航: 完整的HATEOAS实现")
    
    print(f"\n   🌐 API端点总览:")
    print(f"      - GET /api: API发现和导航")
    print(f"      - GET /api/conversations: 对话列表（分页/过滤/排序）")
    print(f"      - GET /api/conversations/{{id}}: 对话详情")
    print(f"      - POST /api/conversations: 创建对话")
    print(f"      - GET /api/stats: API统计信息")
    
    print(f"\n   🎯 RESTful特性验证:")
    print(f"      ✅ 资源导向: 使用名词表示资源")
    print(f"      ✅ 统一接口: 标准HTTP方法和状态码")
    print(f"      ✅ 无状态: 每个请求独立完整")
    print(f"      ✅ 可缓存: 支持HTTP缓存控制")
    print(f"      ✅ 分层系统: 清晰的API架构")
    print(f"      ✅ HATEOAS: 超媒体驱动导航")
else:
    print(f"   ❌ FastAPI不可用，跳过完整API示例")

## 学习总结

### 📚 知识点回顾

In [ ]:
print(f"\n📚 学习总结:")
print("=" * 50)

class RESTfulAPISummary:
    """RESTful API学习内容总结"""
    
    topics_covered = [
        "REST架构原则和约束条件",
        "资源建模和URL设计最佳实践",
        "分页策略和实现（偏移量、游标、键集）",
        "过滤和排序功能实现",
        "API版本控制策略（URL、头部、参数）",
        "速率限制和缓存机制",
        "HATEOAS超媒体导航实现",
        "完整的RESTful API设计模式",
        "LangChain应用API架构设计",
        "生产级API性能优化技巧"
    ]
    
    practical_skills = [
        "能够设计符合REST原则的API架构",
        "实现高效的分页、过滤、排序功能",
        "选择合适的版本控制策略",
        "构建速率限制和缓存系统",
        "实现HATEOAS超媒体导航",
        "设计可扩展的LangChain API",
        "优化API性能和用户体验",
        "遵循API设计最佳实践"
    ]
    
    langchain_api_patterns = [
        "对话管理API（分页历史记录）",
        "文档检索API（高级过滤搜索）",
        "模型管理API（版本控制和缓存）",
        "用户管理API（权限和配额）",
        "流式聊天API（实时响应）",
        "批量处理API（并发优化）",
        "工作流API（任务编排）",
        "分析统计API（性能监控）"
    ]
    
    @classmethod
    def print_summary(cls):
        """打印学习总结"""
        print(f"✅ 已掌握知识点:")
        for i, topic in enumerate(cls.topics_covered, 1):
            print(f"   {i}. {topic}")
        
        print(f"\n🎯 实践技能获得:")
        for i, skill in enumerate(cls.practical_skills, 1):
            print(f"   {i}. {skill}")
        
        print(f"\n🤖 LangChain API模式:")
        for i, pattern in enumerate(cls.langchain_api_patterns, 1):
            print(f"   {i}. {pattern}")
    
    @classmethod
    def design_principles(cls):
        """设计原则总结"""
        print(f"\n💡 RESTful API设计原则:")
        print(f"   1. 资源导向设计：使用名词表示资源，复数表示集合")
        print(f"   2. 统一接口规范：标准HTTP方法和状态码")
        print(f"   3. 无状态通信：每个请求包含完整上下文")
        print(f"   4. 可缓存响应：合理使用HTTP缓存机制")
        print(f"   5. 分层系统架构：清晰的职责分离")
        print(f"   6. 超媒体驱动：HATEOAS导航和发现")
    
    @classmethod
    def performance_optimization(cls):
        """性能优化技巧"""
        print(f"\n🚀 API性能优化技巧:")
        print(f"   1. 分页优化：避免大结果集传输")
        print(f"   2. 缓存策略：减少重复计算和查询")
        print(f"   3. 速率限制：保护服务资源")
        print(f"   4. 异步处理：提升并发性能")
        print(f"   5. 数据压缩：减少网络传输")
        print(f"   6. 连接池：复用数据库连接")
        print(f"   7. CDN加速：静态资源分发")
        print(f"   8. 监控告警：实时性能跟踪")
    
    @classmethod
    def next_steps(cls):
        """下一步学习计划"""
        print(f"\n🎯 下一步学习计划:")
        print(f"   1. 深入学习WebSocket实时通信技术")
        print(f"   2. 掌握异步数据库操作和ORM")
        print(f"   3. 学习Web安全和身份认证机制")
        print(f"   4. 了解微服务架构设计模式")
        print(f"   5. 掌握API文档生成和测试")
        print(f"   6. 学习容器化部署和运维")
        print(f"   7. 构建完整的LangChain生产应用")
        print(f"   8. 实现高可用和负载均衡")

# 打印学习总结
RESTfulAPISummary.print_summary()
RESTfulAPISummary.design_principles()
RESTfulAPISummary.performance_optimization()
RESTfulAPISummary.next_steps()

print(f"\n🎉 RESTful API设计与实现 学习完成!")
print(f"🌐 已掌握构建生产级RESTful API的核心技能")
print(f"⚡ 分页、过滤、版本控制、缓存、限流等功能全面掌握")
print(f"🔗 HATEOAS超媒体导航为API提供了优秀的可发现性")
print(f"🚀 为LangChain应用提供了可扩展的API架构基础")
print(f"🎯 下一步将学习WebSocket实时通信技术")